In [215]:
import sys
import os
import mmh3
import numpy as np
import itertools
import collections
import pandas as pd
import more_itertools
from itertools import tee
import re
#################### Utilities ######################
#hashes a list of strings
def listhash(l,seed):
	val = 0
	for e in l:
		val = val ^ mmh3.hash(e, seed)
	return val 

################### Similarity ######################

docs = {} #dictionary mapping document id to document contents

data = pd.read_csv(r"C:\Users\hasee\Documents\comptools\DTU_CTDS\data\articles1.csv")

data1 = data[:1000]
key = data1["Unnamed: 0"]
data2 = data1["content"]
docs = data2.to_dict()
# docs = data2.tolist()
# g_docs = data2.tolist()
docs = list(zip(data2.tolist())) ## convert to tuple for starmap to work basically getting ['dfdf', 'dddd', 'sss'] => [('dfdf'), ('dddd'), ('sss')]
# print(docs[2])


In [211]:
docs[0]

('WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been

In [206]:
tt=[(1,2,3), (2,), (5,5,5,5,5)]
rr=min(list(map(len, tt)))
rr

1

In [227]:
''' Extension of standard pairwise function to 3-pairwise from the py standard lib'''
def three_pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    # three_pairwise('ABCDEFG') --> ABC BCD CDE DEF EFG FG
    a, b, c = tee(iterable, 3) ## Create three iterators
    next(b, None) ## Advance the second
    next(c, None) ## Advance the third
    next(c, None) ## Advance the third once again. This ensures the third iterator starts at the third element and that we can create the 3-tuple
    return zip(a, b, c) ### Zip everything (1 elem, 2 elem, and 3 elem) concurrently.

def hash_docs(doc, k=50):
    # three_tup=zip
    lst_shingles_h=[]
    # for doc in docs:
    seed=1
    three_tup=three_pairwise(doc.split(" "))

    three_tup=list(three_tup)

    three_tup=[elem for elem in three_tup if '' not in elem]
    
    # for t in three_tup:
    #     if '' or ' ' in t:
    #         print("single q", t)
    #     if " " or "" in t:
    #         print("double ", t)

    # print(set(three_tup))

    # seeds = [seed+i for i in range(0,k)]
    ret_lst=[]

    # for s in seeds:  
    ret_lst.append(np.array(sorted([listhash(shingle, seed) for shingle in three_tup])[:k]))
    
    return ret_lst

''' Returns document as a list of hashes'''
'''
Creates shingles of size q, removing shingles of size < q. Removes duplicates and hashes the result.
'''
def hashed_lst_shingles(my_docs):


    hashed_docs = itertools.starmap(hash_docs, my_docs)
    return hashed_docs, len(my_docs)

# doc = "You and me, we made a vow. For better or for worse. I can't believe you let me down"
# lst_shnigles = hashed_lst_shingles(3, doc)
# print(lst_shnigles)
# [['You', 'and', 'me,'], ['and', 'me,', 'we'], ['me,', 'we', 'made'], ['we', 'made', 'a'], ['made', 'a', 'vow.'], ['a', 'vow.', 'For'], ['vow.', 'For', 'better'], ['For', 'better', 'or'], ['better', 'or', 'for'], ['or', 'for', 'worse.'], ['for', 'worse.', 'I'], ['worse.', 'I', "can't"], ['I', "can't", 'believe'], ["can't", 'believe', 'you'], ['believe', 'you', 'let'], ['you', 'let', 'me'], ['let', 'me', 'down']]

### MinHashing implementation

In [65]:
# def minhash(shingles, seed):
#     # print("what is this>")
#     # print(shingles)
#     return min([listhash(shingle, seed) for shingle in shingles])

# def minhash2(shingles, seed, k):
#     minhashes=[]
#     seeds = [seed+i for i in range(0,k)]
#     for s in seeds:
#         minhashes.append(minhash(shingles, s))
    
#     return minhashes


In [66]:
# def signatures(my_shingles, num_hash, seed=1):
#     # lst_shingles = ex2.shingle(q=shingle_size, in_str=in_str)
#     sig_lst=[]
#     for i, shingle in enumerate(my_shingles):
#         sig = minhash2(shingles=shingle, seed=seed, k=num_hash)
#         sig_lst[i] = sig

#     return sig_lst




In [226]:
# test_docs=list(zip(["You and me, we made a vow. For better or for worse. I can't believe you let me down",
# "Time, space and state. Equal everything explanable.",
# "You and me, we made a vow. For better or for worse. I can't believe you let me down"]))
hashed_shingles, my_len=hashed_lst_shingles(docs)
hashed_shingles = list(hashed_shingles)
# nnn=np.stack(hashed_shingles)
sig_m=np.array(hashed_shingles).T
# sig_m = np.array(list(hashed_shingles)).T
print("sss",sig_m.shape)
sig_m=sig_m.reshape(sig_m.shape[0], sig_m.shape[2])
sig_m.shape


sss (1, 1000)


C:\Users\hasee\AppData\Local\Temp\ipykernel_12596\1348849060.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sig_m=np.array(hashed_shingles).T


IndexError: tuple index out of range

From results of sig matrix above, documents 1 and 3 are very similar (similar hashes)

In [218]:
''' Jaccard similarity'''
def jaccard(s1, s2):
    return len(s1 & s2) / len(s1 | s2)

# score = jaccard(set(sig_m[:, 5]), set(sig_m[:, 6])) # finding similarity b/w documents 5 and 6. remember00palm.txt and remembermeorholy00palm.txt

# print(score)

We see that docs 5 and 6 are quite similar (0.61).

### LSH implementation

In [219]:
''' Implementation of LSH, dividing signature matrix into b band with r rows each'''
def LSH(sig_m, b, r):
    b=20
    r=5
    #b*r=num_hash_funcs

    sim_hashes=[]
    start=0
    for i in range(b):
        sim_hashes.append([listhash(col, seed=i) for col in sig_m[start:start+r,:].T])
        start=i+r

    return sim_hashes

''' Find candidate pairs by checking to see if the hashes match.
Then we check to see that the Jaccard similarity b/w each pair of docs is atleast t. If so, we consider it a candidate pair otherwise not '''
def get_cand_pairs(sim_hashes, t):
    cand_pairs=set()
    for L in sim_hashes:
        dups = collections.defaultdict(list)
        for i, e in enumerate(L):
            dups[e].append(i)
        for k, v in sorted(dups.items()):
            if len(v) >= 2:
                cand_pairs.add(tuple(v))
                # print(k, v)
    cand_pairs=list(cand_pairs)
    filtered_cand_pairs = [pair for pair in cand_pairs if (jaccard(set(sig_m[:, pair[0]]), set(sig_m[:, pair[1]])) > t)]
    return filtered_cand_pairs
    # return cand_pairs

array([-1977450224, -1474616243, -1358494162, -1349483768, -1284103952,
       -1063952611,  -266913181,   307140698,   399421633,   678722671,
         727187198,   790074072,   979649597,  1012655871,  1273310172,
        1657204410,  2064897588])

In [223]:
b=20
r=5
sim_hashes = LSH(sig_m, b=20, r=5)
pairs=get_cand_pairs(sim_hashes, t=(1/b)**(1/r))
pairs

[]

As can be seen using LSH, we find that documents 5 and 6 are similar.